In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
from pandas import DataFrame
import matplotlib.pylab as plt
import os

In [ ]:
PUIdata=os.getenv("PUIDATA") 

In [ ]:
from geopandas import GeoDataFrame
shape_path=PUIdata + "/taxi_zones/taxi_zones.shp"
ct_shape = GeoDataFrame.from_file(shape_path)
ct_shape.index=ct_shape.LocationID

In [ ]:
from fiona.crs import from_epsg
ct_shape.crs = from_epsg(2263) # epsg=4326: lat/on | 26918: NAD83/UTM zone 18N | epsg=2263 is US feet
NYC_Area=ct_shape.to_crs(epsg=2263).geometry.area.sum()/(2.788*10**7) # from ft2 to mi2 
print ('total NYC land area: %s (mi^2)'%str(NYC_Area)[:5]) # from ft2 to mi2 #epsg=2263 is US feet

In [ ]:
yc201604 = pd.read_csv(PUIdata + "/yellow_tripdata_2016-04.csv").dropna(subset=['lpep_pickup_datetime','Trip_distance','pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude'])
yc201604 = yc201604[['lpep_pickup_datetime','Trip_distance','pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude']]

# Inbound

In [1]:
yc201604_ = yc201604[yc201604['dropoff_longitude']<-73.930]
yc201604_ = yc201604_[yc201604_['dropoff_longitude']>-73.979]
yc201604_ = yc201604_[yc201604_['dropoff_latitude']<40.797]
yc201604_ = yc201604_[yc201604_['dropoff_latitude']>40.745]

NameError: name 'yc201604' is not defined

In [ ]:
yc201604_=yc201604_.dropna(axis=0)

In [ ]:
import shapely
# combine lat and lon to one column
yc201604_['lonlat']=zip(yc201604_.pickup_longitude,yc201604_.pickup_latitude)
# Create Point Geometry for based on lonlat column
yc201604_['geometry']=yc201604_[['lonlat']].applymap(lambda x:shapely.geometry.Point(x))
yc201604_.head()

In [ ]:
ct_shape = ct_shape.to_crs({'proj':'longlat', 'ellps':'WGS84', 'datum':'WGS84'})

In [ ]:
a=[]
# check for each citibike stations to see what CT they are belong to
for i in range(len(yc201604_)):
    try:
        a.append(ct_shape[ct_shape.geometry.intersects(yc201604_.iloc[i].geometry)].LocationID.values[0])
    except IndexError:
        a.append(-1)
    print '\r',"%",str((i+1)*100./len(yc201604_))[:4],
yc201604_['LocationID']=a

In [ ]:
yc_inbound=yc201604_[yc201604_.LocationID.isin([229,140,141,262,263])]